In [11]:
# Importing important libraries

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [12]:
# Reading the data into data frame with proper encoding

df = pd.read_csv('data/spam.csv', encoding='latin1')
df = df[['v1', 'v2']]
df.columns = ['label', 'message']

In [13]:
# Converting Textual labels to numbers

df['label'] = df['label'].map({'ham': 0, 'spam': 1})

In [14]:
# Splitting the data frame into feature and target

x = data['message']
y = data['label']

# Splitting the data into training and testing sets

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Vectorization of textual features with TF-IDF

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

In [16]:
# Defining models for prediction

classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC()
}

# Defining parameter grids for GridSearchCV
parameters_cv = {
    'Naive Bayes': {
        'alpha': [0.001, 0.01, 0.1, 1.0]
    },
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear']
    }
}

In [18]:
# Evaluation of all models with GridSearchCV individually

results = {}
for name, clf in classifiers.items():
    grid_search = GridSearchCV(clf, parameters_cv[name], cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(x_train_tfidf, y_train)
    
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test_tfidf)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    results[name] = {
        'Best Params': grid_search.best_params_,
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix(y_test, y_pred),
        'Classification Report': classification_report(y_test, y_pred)
    }

In [19]:
# Printing results for each model

for name, result in results.items():
    print(f"Classifier: {name}")
    print(f"Best Params: {result['Best Params']}")
    print(f"Accuracy: {result['Accuracy']:.4f}")
    print(f"F1 Score: {result['F1 Score']:.4f}")
    print(f"Confusion Matrix:\n{result['Confusion Matrix']}")
    print(f"Classification Report:\n{result['Classification Report']}")
    print("-" * 30)

Classifier: Naive Bayes
Best Params: {'alpha': 0.1}
Accuracy: 0.9830
F1 Score: 0.9827
Confusion Matrix:
[[961   4]
 [ 15 135]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.97      0.90      0.93       150

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115

------------------------------
Classifier: Logistic Regression
Best Params: {'C': 10}
Accuracy: 0.9758
F1 Score: 0.9752
Confusion Matrix:
[[960   5]
 [ 22 128]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       965
           1       0.96      0.85      0.90       150

    accuracy                           0.98      1115
   macro avg       0.97      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

------------